## AI Agent智能应用从0到1定制开发 
## AI Agent Intelligent Application Custom Development from 0 to 1
******
- 此代码为网课《AI Agent智能应用从0到1定制开发》的配套代码，需要注意本套代码建议与网课适配配合食用。
- This code for the online course <AI Agent Intelligent Applications from 0 to 1 custom development> supporting code, need to pay attention to this set of code is recommended with the online course adapted to work with consumption.
- 需要注意由于课程开发周期的原因，langchain版本跨越了3个大版本，部分代码会与视频演示有差别!
- Note that due to the course development cycle, the langchain version spans 3 major releases and some of the code will differ from the video demo!
- 课程地址：https://coding.imooc.com/class/822.html
- Course address: https://coding.imooc.com/class/822.html

### 从环境变量中读取密钥
### Read the key from the environment variable
- 注意：尽量将你的OpenAI Key存储在类似.env文件中，而不是明文暴露在代码里，这是一种基本的安全措施
- Note: Try to store your OpenAI Key in something like an .env file, rather than exposing it explicitly in code, as a basic safety measure!
******

In [1]:

import os
from dotenv import load_dotenv
# Load environment variables from openai.env file
load_dotenv("asset/openai.env")

# Read the OPENAI_API_KEY from the environment
api_key = os.getenv("OPENAI_API_KEY")
api_base = os.getenv("OPENAI_API_BASE")
os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = api_base

### 不同的Memory工具
### Different Memory tools

- 短时记忆：存储在内存中
- Short-term memory: stored in memory
- 构建记忆实体清单
- Constructing a list of memory entities
- 接入知识图谱
- Accessing the Knowledge Graph
- 长对话在内存中的处理方式
- Long conversations handled in memory
- 长时记忆实现方式
- Long-term memory implementation
<hr>

- 内存中的短时记忆
- Short-term memory in memory

In [2]:
from langchain.memory import  ConversationBufferMemory

memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("你好，我是人类！")
memory.chat_memory.add_ai_message("你好，我是AI，有什么可以帮助你的吗？")

memory.load_memory_variables({})

{'history': 'Human: 你好，我是人类！\nAI: 你好，我是AI，有什么可以帮助你的吗？'}

In [3]:
#实现一个最近的对话窗口，超过窗口条数的对话将被删除
# devide a conversation into windows, and only keep the latest k windows
from langchain.memory import  ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=2)

memory.save_context({"input":"你好，我是人类!"},{"output":"你好，我是AI，有什么可以帮助你的吗？"})
memory.save_context({"input":"我想吃鸡肉"},{"output":"好的，我帮你找找鸡肉的做法"})

memory.load_memory_variables({})

{'history': 'Human: 你好，我是人类!\nAI: 你好，我是AI，有什么可以帮助你的吗？\nHuman: 我想吃鸡肉\nAI: 好的，我帮你找找鸡肉的做法'}

- 构建记忆实体概念清单
- Constructing a list of memory entities

In [4]:
from langchain_openai import OpenAI
from langchain.memory import ConversationEntityMemory

llm = OpenAI(
    temperature=0
)

memory = ConversationEntityMemory(llm=llm)
_input = {
    "input":"胡八一和王胖子雪莉杨经常在一起冒险，合称盗墓铁三角."
}
memory.load_memory_variables(_input)
memory.save_context(
    _input,
    {
        "output":"听起来很刺激，我也想加入他们！"
    }
)


In [5]:
memory.entity_cache

['胡八一', '王胖子', '雪莉杨', '冒险', '盗墓铁三角.']

In [6]:
memory.load_memory_variables({"input":"铁三角是谁?"})

{'history': 'Human: 胡八一和王胖子雪莉杨经常在一起冒险，合称盗墓铁三角.\nAI: 听起来很刺激，我也想加入他们！',
 'entities': {}}

- 使用知识图谱构建记忆
- Accessing the Knowledge Graph

In [7]:
from langchain_openai import OpenAI
from langchain.memory import ConversationKGMemory

llm = OpenAI(
    temperature=0
)   

memory = ConversationKGMemory(llm=llm,return_messages=True)

memory.save_context(
    {"input":"帮我找一下tomie"},
    {"output":"对不起请问什么是tomie？"}
)

memory.save_context(
    {"input":"tomie是一个培训讲师"},
    {"output":"好的，我知道了。"}
)

In [8]:
memory.load_memory_variables({"input":"tomie是谁?"})

{'history': [SystemMessage(content='On tomie: tomie is a trainer.')]}

In [9]:
memory.get_current_entities("tomie最喜欢做什么事?")

['tomie']

In [10]:
memory.get_knowledge_triplets("tomie最喜欢打游戏")

[KnowledgeTriple(subject='tomie', predicate='loves playing', object_='games')]

- 长对话在内存中的处理方式：总结摘要以及token计算
- In-memory handling of long conversations: summary abstracts and token computation

In [11]:
from langchain.memory import ConversationSummaryMemory
from langchain_openai import OpenAI

llm = OpenAI(
    temperature=0
)

memory = ConversationSummaryMemory(llm=llm)
memory.save_context(
    {"input":"帮我找一下tomie"},
    {"output":"对不起请问什么是tomie？"}
)
memory.save_context(
    {"input":"tomie是一个培训讲师"},
    {"output":"好的，我知道了。"}
)

In [12]:
memory.load_memory_variables({})

{'history': '\nThe human asks the AI to search for something named "tomie". The AI apologizes and asks for clarification on what "tomie" is. The human explains that it is a trainer. The AI acknowledges and says it understands.'}

In [13]:
messages = memory.chat_memory.messages
#print(messages)
memory.predict_new_summary(messages,"")

'\nThe human requests for help in finding information about "tomie". The AI asks for clarification on who or what "tomie" is. The human explains that "tomie" is a training instructor. The AI acknowledges and states that it now understands.'

In [14]:
#使用ChatMessageHistory来快速获得对话摘要
#use ChatMessageHistory to get conversation summary quickly
from langchain.memory import ConversationSummaryMemory
from langchain.memory import ChatMessageHistory
from langchain_openai import OpenAI

hisiory = ChatMessageHistory()
hisiory.add_user_message("你好，我是人类！")
hisiory.add_ai_message("你好，我是AI小丸子，有什么可以帮助你的吗？")

#memory = ConversationSummaryMemory.from_messages(
#    llm=OpenAI(temperature=0),
#    chat_memory=hisiory,
#    return_messages=True
#)

memory = ConversationSummaryMemory(
    llm=OpenAI(temperature=0),
    return_messages=True,
    buffer="\nThe AI introduces itself as AI Little Maruko and asks if there is anything it can help the human with.",
    chat_memory=hisiory
)

In [15]:
#memory.buffer
memory.load_memory_variables({})

{'history': [SystemMessage(content='\nThe AI introduces itself as AI Little Maruko and asks if there is anything it can help the human with.')]}

In [16]:
# 当对话持续进行且对话内容很多的时候
# 可以使用ConversationSummaryBufferMemory来存储对话摘要
# 这是一种非常有用的方式,它会根据token的数量来自动判断是否需要进行摘要
# 当token数量超过阈值的时候,会自动进行摘要
# 在缓冲区中,会保留最近的k条对话
#比较久的对话会被删除，在删除前会进行摘要

In [17]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain_openai import OpenAI

llm = OpenAI(temperature=0)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=10,
    return_messages=True
)
memory.save_context(
    {"input":"帮我找一下tomie"},
    {"output":"对不起请问什么是tomie？"}
)
memory.save_context(
    {"input":"tomie是一个培训讲师"},
    {"output":"好的，我知道了。"}
)
memory.save_context(
    {"input":"今天他要讲一门关于RAG的课程"},
    {"output":"好的，我知道了。需要RAG的资料吗？"}
)


In [18]:
memory.load_memory_variables({})

{'history': [SystemMessage(content='\nThe human asks the AI to search for "tomie" and the AI apologizes for not knowing what it is. The human clarifies that tomie is a training instructor and the AI acknowledges this. The human then informs the AI that today the instructor will be giving a course on RAG. The AI confirms this and asks if any RAG materials are needed. ')]}

- Conversation Token Buffer使用token长度来决定什么时候刷新内存
- Conversation Token Buffer uses token length to determine when to flush memory

In [19]:
from langchain.memory import ConversationTokenBufferMemory
from langchain_openai import OpenAI
llm = OpenAI(temperature=0)
memory = ConversationTokenBufferMemory(
    llm=llm,
    max_token_limit=150
)
memory.save_context(
    {"input":"帮我找一下tomie"},
    {"output":"对不起请问什么是tomie？"}
)
memory.save_context(
    {"input":"tomie是一个培训讲师"},
    {"output":"好的，我知道了。"}
)
memory.save_context(
    {"input":"今天他要讲一门关于RAG的课程"},
    {"output":"好的，我知道了。需要RAG的资料吗？"}
)
memory.save_context(
    {"input":"不需要资料了，谢谢"},
    {"output":"好的，那我就不打扰你了。"}
)


In [20]:
memory.load_memory_variables({})

{'history': 'Human: 帮我找一下tomie\nAI: 对不起请问什么是tomie？\nHuman: tomie是一个培训讲师\nAI: 好的，我知道了。\nHuman: 今天他要讲一门关于RAG的课程\nAI: 好的，我知道了。需要RAG的资料吗？\nHuman: 不需要资料了，谢谢\nAI: 好的，那我就不打扰你了。'}

### 长时记忆的是实现方式
###  Long-term memory's the way to achieve it

- 通过向量数据库来存储之前的对话内容，有的向量数据库服务还提供自动摘要等，每次对话的时候，都会从向量数据库里查询最相关的文档或历史对话
- Storing previous conversations through a vector database, some vector database services also provide automatic summarization, etc. Each time a conversation occurs, the most relevant documents or historical conversations are queried from the vector database

In [21]:
from langchain_openai import OpenAIEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain_community.vectorstores import FAISS

In [22]:
memory = ConversationBufferMemory()
memory.save_context(
    {"input":"帮我找一下tomie"},
    {"output":"对不起请问什么是tomie？"}
)
memory.save_context(
    {"input":"tomie是一个培训讲师"},
    {"output":"好的，我知道了。"}
)
memory.save_context(
    {"input":"今天他要讲一门关于RAG的课程"},
    {"output":"好的，我知道了。需要RAG的资料吗？"}
)
memory.save_context(
    {"input":"不需要资料了，谢谢"},
    {"output":"好的，那我就不打扰你了。"}
)

vectorstore = FAISS.from_texts(
    memory.buffer.split("\n"),
    OpenAIEmbeddings()
)
FAISS.save_local(vectorstore,"test_faiss")

In [23]:
FAISS.load_local(
    "test_faiss",
    OpenAIEmbeddings(model="text-embedding-3-small"),
    allow_dangerous_deserialization=True,
    ).similarity_search("tomie是什么职业?")

[Document(page_content='Human: 今天他要讲一门关于RAG的课程'),
 Document(page_content='AI: 好的，我知道了。需要RAG的资料吗？'),
 Document(page_content='Human: tomie是一个培训讲师'),
 Document(page_content='AI: 好的，那我就不打扰你了。')]

In [24]:
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.memory import VectorStoreRetrieverMemory
r1 = FAISS.load_local("test_faiss",OpenAIEmbeddings(),allow_dangerous_deserialization=True)
r2 = r1.as_retriever(
    search_kwargs={"k":1}
)
memory2 = VectorStoreRetrieverMemory(
    retriever=r2
)
memory2.load_memory_variables({"prompt":"tomie是什么职业?"})

{'history': 'Human: tomie是一个培训讲师'}